## Implementation of PilotNet

Implement [PilotNet](https://arxiv.org/pdf/1604.07316.pdf) using Keras (with tensorflow backend), with some modifications

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

In [2]:
import keras

from resnet import resnet

from keras.optimizers import SGD
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, ModelCheckpoint
from keras.models import load_model

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5110)


In [3]:
# limit GPU memory usage
# import tensorflow as tf
# from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.8
# set_session(tf.Session(config=config))

### Check data format

In [4]:
%ls ../data/csv/final

v1.csv  v2.csv  v3.csv  v4.csv  v5.csv  v5_train.csv  v5_valid.csv


In [5]:
# define path variables
cur_file = 'v5'
parent_path = os.path.dirname(os.getcwd())

data_path = os.path.join(parent_path, 'data')
img_front_dir_path = os.path.join(data_path, 'img', 'front')
model_path = os.path.join(parent_path, 'model')
log_path = os.path.join(model_path, 'log')


csv_dir_path = os.path.join(data_path, 'csv', 'final')
train_file = os.path.join(csv_dir_path, cur_file + '_train.csv')
valid_file = os.path.join(csv_dir_path, cur_file + '_valid.csv')

# divide by a constant to bound output to [0,100]
OUTPUT_NORMALIZATION = 655.35

In [6]:
df_train = pd.read_csv(os.path.join(data_path, train_file))
print("%d rows" % df_train.shape[0])
df_train.head(3)

208454 rows


,img,wheel-axis,clutch,brake,gas,paddle-left,paddle-right,wheel-button-left-1,wheel-button-left-2,wheel-button-left-3,...,gear-1,gear-2,gear-3,gear-4,gear-5,gear-6,gear-R,front,side_left,side_right
0,97802012_2017_08_08_20_47_09_88.jpg,1328,23221,0,34833,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,97802012_2017_08_08_20_47_09_88_front.jpg,97802012_2017_08_08_20_47_09_88_left.jpg,97802012_2017_08_08_20_47_09_88_right.jpg
1,7a13935b_2017_07_30_15_35_17_02.jpg,288,0,0,65535,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7a13935b_2017_07_30_15_35_17_02_front.jpg,7a13935b_2017_07_30_15_35_17_02_left.jpg,7a13935b_2017_07_30_15_35_17_02_right.jpg
2,3f80cca8_2017_08_08_14_14_12_98.jpg,36,15222,0,38703,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3f80cca8_2017_08_08_14_14_12_98_front.jpg,3f80cca8_2017_08_08_14_14_12_98_left.jpg,3f80cca8_2017_08_08_14_14_12_98_right.jpg


In [7]:
df_val = pd.read_csv(os.path.join(data_path, valid_file))
print("%d rows" % df_val.shape[0])
df_val.head(3)

52114 rows


,img,wheel-axis,clutch,brake,gas,paddle-left,paddle-right,wheel-button-left-1,wheel-button-left-2,wheel-button-left-3,...,gear-1,gear-2,gear-3,gear-4,gear-5,gear-6,gear-R,front,side_left,side_right
0,3f80cca8_2017_08_08_14_52_41_43.jpg,140,17287,0,35349,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3f80cca8_2017_08_08_14_52_41_43_front.jpg,3f80cca8_2017_08_08_14_52_41_43_left.jpg,3f80cca8_2017_08_08_14_52_41_43_right.jpg
1,7d590ce8_2017_08_07_13_54_16_42.jpg,-2209,27091,0,53410,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7d590ce8_2017_08_07_13_54_16_42_front.jpg,7d590ce8_2017_08_07_13_54_16_42_left.jpg,7d590ce8_2017_08_07_13_54_16_42_right.jpg
2,15be80cb_2017_07_28_23_24_53_41.jpg,-2625,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,15be80cb_2017_07_28_23_24_53_41_front.jpg,15be80cb_2017_07_28_23_24_53_41_left.jpg,15be80cb_2017_07_28_23_24_53_41_right.jpg


###  Predict steering angle using only the front image

In [8]:
def img_to_arr(p):
    with image.load_img(p) as img:
        img = image.img_to_array(img)
    return img

# values computed from dataset sample.
def normalize(img):
    img[:,:,0] -= 89.5761
    img[:,:,0] /= 58.4214

    img[:,:,1] -= 97.5966
    img[:,:,1] /= 61.7917

    img[:,:,2] -= 88.3135
    img[:,:,2] /= 68.2043
    
    return img

In [9]:
# define generator that loops through the data
def generator(df, batch_size, img_shape, should_shuffle):
    print('shuffle')
    img_list = df['front']
    wheel_axis = df['wheel-axis']
    
    # create empty batch
    batch_img = np.zeros((batch_size,) + img_shape)
    batch_label = np.zeros((batch_size, 1))
    
    index = 0
    while True:
        for i in range(batch_size):
            img_name = img_list[index]
            arr = img_to_arr(os.path.join(img_front_dir_path, img_name))
            
            batch_img[i] = normalize(arr)
            batch_label[i] = wheel_axis[index] / OUTPUT_NORMALIZATION
            
            index += 1
            
            if index == len(img_list):
                index = 0 
                if should_shuffle:
                    df = shuffle(df)
                    img_list = df['front']
                    wheel_axis = df['wheel-axis']
            
        yield batch_img, batch_label
    

In [10]:
input_shape = img_to_arr(os.path.join(img_front_dir_path, df_train['front'][0])).shape
batch_size = 45
train_steps = (df_train.shape[0] / batch_size) + 1
val_steps = (df_val.shape[0] / batch_size) + 1

print("input_shape: %s, batch_size: %d, train_steps: %d, val_steps: %d" % 
      (input_shape, batch_size, train_steps, val_steps))

input_shape: (3, 341, 562), batch_size: 45, train_steps: 4633, val_steps: 1159


In [11]:
train_batch = generator(df_train, batch_size, input_shape, True)
val_batch = generator(df_val, batch_size, input_shape, False)

### Define model

In [12]:
# test resnet
model = resnet(input_shape=input_shape, filter_size=64)
sgd = SGD(lr=1e-3, decay=1e-4, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='mse')
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, 341, 562)   0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, 3, 347, 568)   0           input_1[0][0]                    
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 64, 171, 281)  9472        zero_padding2d_1[0][0]           
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 64, 171, 281)  256         conv1[0][0]                      
___________________________________________________________________________________________

bn3b_branch2b (BatchNormalizatio (None, 128, 43, 70)   512         res3b_branch2b[0][0]             
____________________________________________________________________________________________________
activation_12 (Activation)       (None, 128, 43, 70)   0           bn3b_branch2b[0][0]              
____________________________________________________________________________________________________
add_4 (Add)                      (None, 128, 43, 70)   0           activation_12[0][0]              
                                                                   activation_10[0][0]              
____________________________________________________________________________________________________
activation_13 (Activation)       (None, 128, 43, 70)   0           add_4[0][0]                      
____________________________________________________________________________________________________
res4a_branch2a (Conv2D)          (None, 256, 22, 35)   295168      activation_13[0][0]     

Total params: 11,196,353
Trainable params: 11,186,625
Non-trainable params: 9,728
____________________________________________________________________________________________________


In [12]:
# or load from saved model
model = load_model(os.path.join(model_path, 'v5-resnet_v1-010-0.27778.h5'))

### Define callback for training

In [13]:
# define callbacks
cur_model = cur_file + '-resnet_v1'
csv_logger = CSVLogger(os.path.join(log_path, cur_model + '.log'))

model_file_name= os.path.join(model_path, cur_model + '-{epoch:03d}-{val_loss:.5f}.h5')
checkpoint = ModelCheckpoint(model_file_name, verbose=0, save_best_only=True)

### Train

In [14]:
model.fit_generator(train_batch, 
                    train_steps, 
                    initial_epoch=5,
                    epochs=25, 
                    verbose=1, 
                    callbacks=[csv_logger, checkpoint], 
                    validation_data=val_batch, 
                    validation_steps=val_steps)

Epoch 6/25
4633/4633 [==============================] - 8801s - loss: 1.3172 - val_loss: 1.8076
Epoch 7/25
4633/4633 [==============================] - 11619s - loss: 0.6320 - val_loss: 0.4160
Epoch 8/25
4633/4633 [==============================] - 10119s - loss: 0.9152 - val_loss: 1.2214
Epoch 9/25
4633/4633 [==============================] - 12252s - loss: 0.3974 - val_loss: 0.3197
Epoch 10/25
4633/4633 [==============================] - 12009s - loss: 0.3646 - val_loss: 0.3917
Epoch 11/25
4633/4633 [==============================] - 12856s - loss: 0.3339 - val_loss: 0.2778
Epoch 12/25
4633/4633 [==============================] - 12997s - loss: 0.3265 - val_loss: 0.2985
Epoch 13/25
4633/4633 [==============================] - 11752s - loss: 0.3371 - val_loss: 1.3476
Epoch 14/25
 206/4633 [>.............................] - ETA: 10725s - loss: 0.3638

KeyboardInterrupt: 

In [16]:
# save model if necessary
# model.save(os.path.join(model_path, 'v5-resnet_v1-005-1.76802.h5'))

In [ ]:
# if needed, decrease lr
# sgd = SGD(lr=1e-4, decay=1e-5, momentum=0.9, nesterov=True)
# model.compile(optimizer=sgd, loss="mse")

In [ ]:
# train again
model.fit_generator(train_batch, 
                    train_steps, 
                    initial_epoch=11,
                    epochs=25, 
                    verbose=1, 
                    callbacks=[csv_logger, checkpoint], 
                    validation_data=val_batch, 
                    validation_steps=val_steps)

Epoch 12/25
shuffle
4633/4633 [==============================] - 5317s - loss: 0.2865 - val_loss: 2.8598
Epoch 13/25
4633/4633 [==============================] - 11012s - loss: 0.9030 - val_loss: 0.2572
Epoch 14/25
4633/4633 [==============================] - 7981s - loss: 1.9710 - val_loss: 1.8519
Epoch 15/25
2268/4633 [=============>................] - ETA: 3499s - loss: 2.1239